## Secciones de Inicializacion

### Librerias

In [173]:
import os
#import sys
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

from referencias.lista_enlazada import listaEnlazada as referenciaLista
from referencias.arbol_disc import bst as referenciaArbol
#import numpy as np
import ipywidgets as widgets # for the interactive user interface
from ipywidgets import *
from IPython.display import display, clear_output, SVG, HTML
#from gvanim import Animation
#from gvanim.jupyter import interactive

#import graphviz as gv # for visualizing a tree using Digraph
from graphviz import Digraph, Graph
#import pydot
import matplotlib.pyplot as plt
#import pandas as pd
import random
import string

#from ipyfilechooser import FileChooser
import json
#import time

import  tkinter as tk
from tkinter import filedialog
import shutil

import importlib.util
import functools

### Inicializacion de la Interfaz

In [174]:
out, out1, out2, out3 = Output(), Output(), Output(), Output()
tab = Tab(children = [out1, out2, out3],
          layout=Layout(width='99%', height='auto'))
tab.set_title(0, 'Listas Enlazadas')
tab.set_title(1, 'BST')
tab.set_title(2, 'Grafos')

form_item_layout = Layout(
    #display='flex',
    justify_content='center'
)
col = "black"    

### Metodos de soporte

In [175]:
VALIDATION_STATES = {0:'WARNING:', 1:'SUCCESSFUL:', -1:'FAILED:'}
colorPointer = 'grey'
colorRight = 'blue'
colorLeft = 'green'
colorHigh = 'red'

def defaultfunction(id1, id2):
    if type(id1) != type(id2):
        id1 = str(id1)
        id2 = str(id2)
    if id1 > id2:
        return 1
    elif id1 < id2:
        return -1
    return 0

def create_n_random(n):
    nodos = list()
    for i in range(n*n):
        num = random.randint(1,50)
        if num not in nodos:
            nodos.append(num)  
        if len(nodos) == n:
            break
    return nodos 

def createTuples(nodos, tipo):
    tuples = list()
    if tipo == 1:
        ids = range(0,len(nodos)+2)
        tuples.append((ids[0],'First'))
        for i in range(1,len(ids)-1):
            tuples.append((ids[i],nodos[i-1]))
        if len(nodos) > 0:
            tuples.append((ids[len(nodos)+1],'None'))
    else:
        ids = range(0,len(nodos)+3)
        tuples.append((ids[0],'First'))
        if len(nodos) > 0:
            tuples.append((ids[1],'None'))
            for i in range(2,len(ids)-1):
                tuples.append((ids[i],nodos[i-2]))
            tuples.append((ids[len(nodos)+2],'None'))
    return tuples

def get_random_string():
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(20))
    return result_str


## Metodos Graficos   

### Metodo Grafico Listas Enlazadas

In [176]:
def displayList(estructura, tipo, nodosX=list()):
    try:
        nodos = estructura.getNodeValues()
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    dot = Digraph()
    
    dot.graph_attr = {
        'rankdir': 'LR',
        'center': 'true',
        'size':'14,5',
        'ratio':'fill'
    }
    
    tuples = createTuples(nodos, tipo)
    
    for i,j in tuples:
        if j == 'First' or j == 'None':
            dot.node(name=str(i), label=str(j), shape="square", color='white')
        elif j in nodosX:
            dot.node(name=str(i), label=str(j), shape="square", color=colorHigh)
        else:
            dot.node(name=str(i), label=str(j), shape="square")
    if len(nodos) == 0:
        dot.node(name='-1', label='', shape="square", color='white')
        dot.edge(str(tuples[0][0]), str('-1'), color=colorPointer)
    else:
        for i in range(1,len(tuples)):
            if tuples[i-1][1] == 'First' and tuples[i][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorPointer)
            elif tuples[i-1][1] == 'First' and tuples[i][1] == 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i+1][0]), color=colorPointer)
            elif tuples[i-1][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorRight)
        if tipo == 2:
            for i in range(1, len(tuples)-1):
                if tuples[i][1] != 'None':
                    dot.edge(str(tuples[i][0]), str(tuples[i-1][0]), color = colorLeft)
    display(dot)

### Metodo Grafico Arboles

In [177]:
from DISClib.ADT import orderedmap as omap
from DISClib.ADT import list as lt
from DISClib.Algorithms.Trees import traversal as tr

class BST_grafico():
    def __init__(self):
        self.estructura = omap.newMap(omaptype = 'BST', comparefunction = defaultfunction)

    def addNode_byValue(self, infoNodo):    
        self.estructura = omap.put(self.estructura, infoNodo, infoNodo)

    def getNodeValues(self):
        lst = list()
        if not omap.isEmpty(self.estructura):
            iter = lt.iterator(tr.preorder(self.estructura))
            for i in iter:
                lst.append(i)
        return lst

    def visualize(self, lst = []):
        tree = self.getNodeValues()
        if len(tree) > 0:
            tree = omap.get(self.estructura, tree[0])
            # Recursively add nodes and edges
            def add_nodes_edges(tree, dot=None):
                col = "black"
                # Create Graphviz Digraph 
                if dot is None:
                    dot = Digraph()
                    dot.graph_attr = {
                        'rankdir': 'TB',
                        'center': 'true',
                        'size':'14,5',
                        'ratio':'auto'
                    }
                    dot.node(name='root', label='root', color = 'white', shape="circle", fixedsize="True", width="0.4")
                    if (lst != [] and tree['value'] in lst):
                        dot.node(name=str(tree['value']), label=str(tree['value']), color = colorHigh, shape="circle", fixedsize="True", width="0.4")
                    else:
                        dot.node(name=str(tree['value']), label=str(tree['value']), color = col, shape="circle", fixedsize="True", width="0.4")
                    dot.edge('root', str(tree['value']), color=colorPointer)      
                
                # Add nodes recursively
                if tree['left'] != None:
                    if (lst != [] and tree['left']['value'] in lst):
                        col = "red"  
                    dot.node(name=str(tree['left']['value']), label=str(tree['left']['value']),
                            color = col, shape="circle", fixedsize="True", width="0.4")
                    col = "black"
                    dot.edge(str(tree['value']), str(tree['left']['value']),color=colorLeft)
                    dot = add_nodes_edges(tree['left'], dot=dot)
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                            color = 'white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux,color=colorLeft)
                    #dot = add_nodes_edges(tree['left'], dot=dot)
                                
                if tree['right'] != None:
                    if (lst != [] and tree['right']['value'] in lst): 
                        col = "red" 
                    dot.node(name=str(tree['right']['value']), label=str(tree['right']['value']), 
                            color = col, shape="circle", fixedsize="True", width="0.4")
                    col = "black"
                    dot.edge(str(tree['value']), str(tree['right']['value']), color=colorRight)
                    dot = add_nodes_edges(tree['right'], dot=dot)            
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                    color = 'white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux,color=colorRight)
                    #dot = add_nodes_edges(tree['left'], dot=dot)
                return dot        
            return add_nodes_edges(tree)                   

        else:
            dot = Digraph()
            dot.node(name='root', label='root', color = 'white', shape="circle", fixedsize="True", width="0.4")
            dot.node(name='-1', label='', shape="square", color='white')
            dot.edge('root', str(-1), color=colorPointer)
            return dot

In [178]:
def displayBST(estructura, nodosX = []):
    try:
        nodos = estructura.getNodeValues('Preorder')
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    bst = BST_grafico()
    for i in nodos:
        bst.addNode_byValue(i)
    dot = bst.visualize(nodosX)
    
    #dot2 = Digraph()
    #dot2.node(name='colores', label='Colores', color = 'black', shape="none", fixedsize="True", width="1")
    #dot2.node(name='indicativoLeft2', label='', color = 'white', shape="none", fixedsize="True", width="1")
    #dot2.edge('colores', 'indicativoLeft2', color=colorLeft, label='left')
    #dot2.node(name='indicativoRight2', label='', color = 'white', shape="none", fixedsize="True", width="1")
    #dot2.edge('colores', 'indicativoRight2', color=colorRight, label='right')
    #dot2.subgraph(dot)
    #display(dot2)
    display(dot)

### Metodo Grafico Grafos

## Metodos de validacion

### Metodos de Validacion Listas Enlazadas

In [179]:
def validar_lista_anadir(init_test, end_test, nodo, tipo):
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    return state_val, end_val, comment

def validar_lista_eliminar(init_test, end_test, nodo, tipo, ans):
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion
      
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
            end_val = sorted(end_val, key=functools.cmp_to_key(defaultfunction))
            if(end_test == end_val):
                comment = 'Se eliminó una de las instancias del elemento "'+ str(nodo) + '"'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" No se eliminó satisfactoriamente, se eliminó un elemento diferente'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2   
    
    return state_val, end_val, comment

def validar_lista_encontrar(nodos, nodo, ans): #TODO: corregir validacion
    if nodo in nodos:
        ans_val = True
    else:
        ans_val = False
    
    if ans == ans_val:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" si se encuentra en la lista'
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en la lista'
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" si se encuentra en la lista pero el método isNodeVale() reporta que no se encuentra'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, ans_val, comment

def validar_lista_adyacentes(init_test, listaAdj, nodo, tipo):
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    listaAdj_val = structure_ref.findAdjacentNode(nodo)
    
    listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '
        
    if listaAdj == listaAdj_val:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + txt[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los adyacentes del elemento\nSe esperaban los elementos: ' + txt[:-2]
        txt = ''
        for i in listaAdj:
            txt = txt + str(i) + ', '
        comment = comment + '\nSe obtuvo: ' + txt[:-2]
        state_val = VALIDATION_STATES[-1]
    
    exists = structure_ref.isNodeValue(nodo)
        
    return state_val, listaAdj_val, comment, exists

def validar_lista_crear(nodos, st_nodos, tipo):
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Nodos: ' + txt_nodos[:-2]
            else:
                comment = 'Nodos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
        
    return state_val, comment

### Metodos de Validacion Arboles

In [180]:
def validar_bst_crear(nodos, st_nodos):
    structure_ref = referenciaArbol()
    for i in nodos:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    nodos = structure_ref.getNodeValues("Preorder")
    
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Nodos: ' + txt_nodos[:-2]
            else:
                comment = 'Nodos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
    
    return state_val, comment

def validar_bst_anadir(init_test, end_test, nodo):
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues("Preorder") #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        if len(init_test) == len(end_val):
            comment = 'El elemento "'+ str(nodo) + '" ya se encontraba en el bst'
            state_val = VALIDATION_STATES[1]
        else:
            for i in range(len(end_test)):
                if end_test[i] != end_val[i]:
                    sameOrder = False
                    break                       
            if sameOrder:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
                state_val = VALIDATION_STATES[1]
            else:
                test = set(end_test)
                val = set(end_val)
                if test == val:
                    comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                    state_val = VALIDATION_STATES[0]
                else:
                    comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                    state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, end_val, comment

def validar_bst_eliminar(init_test, end_test, nodo, ans):
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues("Preorder") #Resultado de validacion
      
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder and end_test != init_test:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" No se pertenece al BST'
            state_val = VALIDATION_STATES[0]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La estructura no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La estructura presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2   
    return state_val, end_val, comment

def validar_bst_encontrar(nodos, nodo, ans):
    if nodo in nodos:
        ans_val = True
    else:
        ans_val = False
    
    if ans == ans_val:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" si se encuentra en la lista'
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en la lista'
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" si se encuentra en la estructura pero el método isNodeVale() reporta que no se encuentra'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, ans_val, comment

def validar_bst_adyacentes(init_test, listaAdj, nodo):
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    listaAdj_val = structure_ref.findAdjacentNode(nodo)   
    
    listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '

    if listaAdj == listaAdj_val:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + txt[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los adyacentes del elemento\nSe esperaban los elementos: ' + txt[:-2]
        txt = ''
        for i in listaAdj:
            txt = txt + str(i) + ', '
        comment = comment + '\nSe obtuvo: ' + txt[:-2]
        state_val = VALIDATION_STATES[-1]
    
    exists = structure_ref.isNodeValue(nodo)
    
    return state_val, listaAdj_val, comment, exists

def validar_bst_darNodos(init_test, nodos, orden):
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    nodos_val = structure_ref.getNodeValues(orden)
    
    txt_val = ''
    for i in nodos_val:
        txt_val = txt_val + str(i) + ', '
        
    txt_test = ''
    for i in nodos:
        txt_test = txt_test + str(i) + ', '
    
    if nodos_val == nodos:
        comment = 'Se obtuvo de manera exitosa los elementos en ' + orden + '\nElementos: ' + txt_val[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        nodos_val = sorted(nodos_val, key=functools.cmp_to_key(defaultfunction))
        nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
        if nodos == nodos_val:
            comment = 'No se obtuvieron los elementos en el orden correcto (' + orden + '):'
            comment = comment + '\nSe esperaba: ' + txt_val[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_test[:-2]
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Hay una cantidad diferente de elementos de la esperada (' + orden + '):'
            comment = comment + '\nSe esperaba: ' + txt_val[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_test[:-2]
            state_val = VALIDATION_STATES[-1]            
    return state_val, nodos_val, comment

### Metodos de Validacion Grafos

## Metodos de Enlace

### Metodos de Enlace Lista Enlazada

In [181]:
def crearListaEnlazada(tipo, file, init, data={}):  #err
    """
    Crea una lista enlazada

    Args:
        tipo: Sencilla o Doble
        init: Vacia o Random
    Returns:
        -
    Raises:
        Exception
    """
    
    if tipo == 1: txt = 'Sencilla - ' + init
    else: txt = 'Doble - ' + init
    
    txtNodos = ''
    try:
        estructura = file.listaEnlazada(tipo)
    except:
        e = "\tProblema al crear la lista enlazada, método listaEnlazada()"
        raise Exception(e)
    long = 0
    nodos = list()
    if init == 'Random':
        txtNodos = '\tValores: '
        long = random.randint(5,15)
        nodos = create_n_random(long)
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    elif init == 'Estática' or init == 'Archivo':
        try:
            txtNodos = '\tValores: '
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    displayList(estructura, tipo) 
    
    try:
        st_nodos = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    state_val, comment = validar_lista_crear(nodos, st_nodos, tipo)
    
    print('Crear Lista Enlazada: ' + txt)
    print(state_val, comment)
    return estructura

def anadirNodoLista(estructura, tipo, nodo):        #err
    """
    Añade un nodo a lista

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_lista_anadir(init_test, end_test, nodo, tipo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo)
    print('Añadir elemento')
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def eliminarNodoLista(estructura, tipo, nodo):      #err
    """
    Elimina un nodo a lista

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_lista_eliminar(init_test, end_test, nodo, tipo, ans) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
    out.clear_output()
    displayList(estructura, tipo)
    print('Eliminar elemento')
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def encontrarNodoLista(estructura, tipo, nodo):     #err
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues()
        state_val, ans_val, comment = validar_lista_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans_val:
        displayList(estructura, tipo, lista)
    else:    
        displayList(estructura, tipo)
    
    print(state_val, comment)

def findAdjacentNodeLista(estructura, tipo, nodo):  #err
    """
    Encuentra los adyacentes de un nodo a lista

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
        state_val, listaAdj_val, comment, exists = validar_lista_adyacentes(init_test, listaAdj, nodo, tipo) 
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo, listaAdj)
    
    if not exists:
        print('Encontrar Adyacentes\n\tEl elemento "'+str(nodo)+ '" no existe en la lista')
    else:
        print('Encontrar Adyacentes')
        print(state_val, comment)

def darTodosLosNodos(estructura, tipo):
    """
    Retorna una lista con todos los nodos de la lista

    Args:
        estructura: lista enlazada
    Returns:
        Lista de los elementos de la lista
    Raises:
        Exception
    """
    try:
        nodos = estructura.getNodeValues()
    except:
        raise Exception('\tProblema al obtener todos los elemento, método getNodeValues()')
    out.clear_output()
    displayList(estructura, tipo)
    txt = ''
    for i in nodos:
        txt = txt + str(i) + ', '
    
    print('Encontrar Todos')
    print('\tTotal Nodos:', str(len(nodos)))
    print('\tValores:', txt[:-2])


### Metodos de Enlace Arboles

In [182]:
def crearBST(init, file, data={}):
    """
    Crea un arbol BST

    Args:
        file: Estructura cargada del estudiante
        init: Vacia, Random, Estática, Archivo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        #estructura = file.bst()
        estructura = file.bst()
    except:
        e = "\tProblema al crear el arbol BST, método bst()"
        raise Exception(e)
    long = 0
    nodos = list()
    if init == 'Random':
        long = random.randint(5,10)
        nodos = create_n_random(long)
        for i in nodos:
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    elif init == 'Estática' or init == 'Archivo':
        try:
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    displayBST(estructura) 
    
    try:
        st_nodos = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    state_val, comment = validar_bst_crear(nodos, st_nodos)
    
    print('Crear BST ' + init + ':', state_val)
    print(comment)
    return estructura

def anadirNodoBST(estructura, nodo):
    """
    Añade un nodo al BST

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)
        end_test = estructura.getNodeValues("Preorder")
        state_val, end_val, comment = validar_bst_anadir(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura)
    print('Añadir elemento')
    
    print(state_val + ':', comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def eliminarNodoBST(estructura, nodo):
    """
    Elimina un nodo al BST

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues("Preorder")
        state_val, end_val, comment = validar_bst_eliminar(init_test, end_test, nodo, ans) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
    out.clear_output()
    displayBST(estructura)
    print('Eliminar elemento')
    
    print(state_val + ':', comment)
    if state_val == VALIDATION_STATES[-1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def encontrarNodoBST(estructura, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues("Preorder")
        state_val, ans_val, comment = validar_bst_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans_val:
        displayBST(estructura, lista)
    else:    
        displayBST(estructura)
    print('Existe elemento')
    print(state_val + ':', comment)
    
def findAdjacentNodoBST(estructura, nodo):
    """
    Encuentra los adyacentes de un nodo en el bst

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
        state_val, listaAdj_val, comment, exists = validar_bst_adyacentes(init_test, listaAdj, nodo) 
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura, listaAdj)
    
    if not exists:
        print('Encontrar Adyacentes\n\tEl elemento "'+str(nodo)+ '" no existe en el bst')
    else:
        print('Encontrar Adyacentes')
        print(state_val, comment)

def listarNodosBST(estructura, orden):
    """
    Encuentra los adyacentes de un nodo en el bst

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        nodos = estructura.getNodeValues(orden)
        state_val, nodos_val, comment = validar_bst_darNodos(init_test, nodos, orden) 
    except:
        e = '\tProblema al listar todos los elementos, método getNodeValues()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura)
    print('Listar todos los nodos')
    print(state_val, comment)

### Metodos de Enlace Grafos

## Layout: Opciones

### Opciones Lista Enlazada

In [183]:
import this

listaType = widgets.RadioButtons(
    options = ['Sencilla', 'Doble'],
    disabled = False
)

listaInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática' ,'Archivo'],
    disabled = False
)

listaOrden = widgets.RadioButtons(
    options = ['Preorder', 'Inorder', 'Postorder'],
    disabled = False
)

listaInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del nodo',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crear = Button(description='Crear', width='extended')
def on_button_crear_clicked(b):
    with out:
        out.clear_output()
        if listaType.value == 'Sencilla':  this.type = 1
        else: this.type = 2
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            else:
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista')
            print(e)
btn_crear.on_click(on_button_crear_clicked)

btn_addNode = Button(description='Añadir', width='extended')
def on_button_addNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                anadirNodoLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNode.on_click(on_button_addNode_clicked)

btn_deleteNode = Button(description='Eliminar', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                eliminarNodoLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Validar', width='extended')
def on_button_findNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                encontrarNodoLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                findAdjacentNodeLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_todosNodos = Button(description='Encontrar Todos', width='extended')
def on_button_todosNodos_clicked(b):
    with out:    
        try:
            try:
                this.estructura
            except:
                raise Exception("\tLa estructura no ha sido creada")
            darTodosLosNodos(this.estructura, this.type)
        except Exception as e:
            print('Hubo un problema al intentar encontrar todos los nodos')
            print(e)
btn_todosNodos.on_click(on_button_todosNodos_clicked)

### Opciones Arbol Binario

In [184]:
btn_crearArbolOption = Button(description='Crear', width='extended')
def on_button_crearArbolOption_clicked(b):
    with out:
        out.clear_output()
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/bst_1.json' 
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearBST(listaInit.value, this.file, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearBST(listaInit.value, this.file, data)
            else:
                this.estructura = crearBST(listaInit.value, this.file)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista')
            print(e)
btn_crearArbolOption.on_click(on_button_crearArbolOption_clicked)

btn_addNodeArbolOption = Button(description='Añadir', width='extended')
def on_button_addNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                anadirNodoBST(this.estructura, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeArbolOption.on_click(on_button_addNodeArbolOption_clicked)

btn_deleteNodeArbolOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                eliminarNodoBST(this.estructura, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeArbolOption.on_click(on_button_deleteNodeArbolOption_clicked)

btn_findNodeArbolOption = Button(description='Validar', width='extended')
def on_button_findNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                encontrarNodoBST(this.estructura, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeArbolOption.on_click(on_button_findNodeArbolOption_clicked)

btn_findAdjNodeArbolOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                findAdjacentNodoBST(this.estructura, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNodeArbolOption.on_click(on_button_findAdjNodeArbolOption_clicked)

btn_todosNodosArbol = Button(description='Listar nodos', width='extended')
def on_button_todosNodosArbol_clicked(b):
    with out:
        out.clear_output()
        try:
            try:
                this.estructura
            except:
                raise Exception("\tLa estructura no ha sido creada")
            listarNodosBST(this.estructura, listaOrden.value)
        except Exception as e:
            print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
            print(e)
btn_todosNodosArbol.on_click(on_button_todosNodosArbol_clicked)

### Opciones Grafo

In [185]:
btn_crearGrafoOption = Button(description='Crear', width='extended')
def on_button_crearGrafoOption_clicked(b):
    with out:
        print('Crear Grafo')
btn_crearGrafoOption.on_click(on_button_crearGrafoOption_clicked)

btn_addNodeGrafoOption = Button(description='Añadir', width='extended')
def on_button_addNodeGrafoOption_clicked(b):
    with out:
        print('Add node Grafo')
btn_addNodeGrafoOption.on_click(on_button_addNodeGrafoOption_clicked)

btn_deleteNodeGrafoOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeGrafoOption_clicked(b):
    with out:
        print('delete node Grafo')
btn_deleteNodeGrafoOption.on_click(on_button_deleteNodeGrafoOption_clicked)

btn_findNodeGrafoOption = Button(description='Validar', width='extended')
def on_button_findNodeGrafoOption_clicked(b):
    with out:
        print('Find node Grafo')
btn_findNodeGrafoOption.on_click(on_button_findNodeGrafoOption_clicked)

btn_findAdjNodeGrafoOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeGrafoOption_clicked(b):
    with out:
        print('find adj Grafo')
btn_findAdjNodeGrafoOption.on_click(on_button_findAdjNodeGrafoOption_clicked)

## Interfaz

### Layout Lista Enlazada

In [186]:
# Panel de Opciones Lista Enlazada
items_optionsCreateList = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Lista Enlazada')], layout=form_item_layout),
    Box([Label(value='Tipo:')]),
    Box([listaType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crear]),
]
items_optionsAddNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
]
items_optionsDeleteNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
]
items_optionsFindNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
]
items_optionsFindAdjNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
]
items_optionsFindAllNodes = [
    Box([Label(value=' ')]),
    #Box([Label(value='Encontrar Todos los Elementos')], layout=form_item_layout),
    Box([btn_todosNodos], layout=form_item_layout),
]

layoutOptionsCreate = Layout(
    display='none',
    #border = 'solid 1px',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodes = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateList = Box(items_optionsCreateList, layout=layoutOptionsCreate)
optionsAddNode = Box(items_optionsAddNode, layout=layoutOptionsAddNode)
optionsDeleteNode = Box(items_optionsDeleteNode, layout=layoutOptionsDeleteNode)
optionsFindNode = Box(items_optionsFindNode, layout=layoutOptionsFindNode)
optionsFindAdjNode = Box(items_optionsFindAdjNode, layout=layoutOptionsFindAdjNode)
optionsFindAllNodes = Box(items_optionsFindAllNodes, layout=layoutOptionsFindAllNodes)


### Layout Arbol

In [187]:
# Panel de Opciones Arbol
items_optionsCreateArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Árbol Binario')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearArbolOption]),
]
items_optionsAddNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeArbolOption], layout=form_item_layout),
]
items_optionsDeleteNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeArbolOption], layout=form_item_layout),
]
items_optionsFindNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeArbolOption], layout=form_item_layout),
]
items_optionsFindAdjNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeArbolOption], layout=form_item_layout),
]

items_optionsFindAllNodesArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Listar todos los nodos')], layout=form_item_layout),
    Box([Label(value='Orden:')]),
    Box([listaOrden]),
    Box([btn_todosNodosArbol]),
]

layoutOptionsCreateArbol = Layout(
    display='none',
    #border = 'solid 1px',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodesArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateArbol = Box(items_optionsCreateArbol, layout=layoutOptionsCreateArbol)
optionsAddNodeArbol = Box(items_optionsAddNodeArbol, layout=layoutOptionsAddNodeArbol)
optionsDeleteNodeArbol = Box(items_optionsDeleteNodeArbol, layout=layoutOptionsDeleteNodeArbol)
optionsFindNodeArbol = Box(items_optionsFindNodeArbol, layout=layoutOptionsFindNodeArbol)
optionsFindAdjNodeArbol = Box(items_optionsFindAdjNodeArbol, layout=layoutOptionsFindAdjNodeArbol)
optionsFindAllNodesArbol = Box(items_optionsFindAllNodesArbol, layout=layoutOptionsFindAllNodesArbol)

### Layout Grafo

In [188]:
# Panel de Opciones Grafo
items_optionsCreateGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Grafo')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearGrafoOption]),
]
items_optionsAddNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeGrafoOption], layout=form_item_layout),
]
items_optionsDeleteNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindAdjNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeGrafoOption], layout=form_item_layout),
]

layoutOptionsCreateGrafo = Layout(
    display='none',
    #border = 'solid 1px',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateGrafo = Box(items_optionsCreateGrafo, layout=layoutOptionsCreateGrafo)
optionsAddNodeGrafo = Box(items_optionsAddNodeGrafo, layout=layoutOptionsAddNodeGrafo)
optionsDeleteNodeGrafo = Box(items_optionsDeleteNodeGrafo, layout=layoutOptionsDeleteNodeGrafo)
optionsFindNodeGrafo = Box(items_optionsFindNodeGrafo, layout=layoutOptionsFindNodeGrafo)
optionsFindAdjNodeGrafo = Box(items_optionsFindAdjNodeGrafo, layout=layoutOptionsFindAdjNodeGrafo)

### Build Layouts

In [189]:
allOptions = [optionsCreateList,
            optionsAddNode,
            optionsDeleteNode,
            optionsFindNode,
            optionsFindAdjNode,
            optionsFindAllNodes,
            optionsCreateArbol,
            optionsAddNodeArbol,
            optionsDeleteNodeArbol,
            optionsFindNodeArbol,
            optionsFindAdjNodeArbol,
            optionsFindAllNodesArbol,
            optionsCreateGrafo,
            optionsAddNodeGrafo,
            optionsDeleteNodeGrafo,
            optionsFindNodeGrafo,
            optionsFindAdjNodeGrafo]

def hideShowOptions(option):
    for i in allOptions:
        i.layout.visibility = 'hidden'
        i.layout.display = 'none'
    listaInputNode.value = ''
       
    option.layout.display = 'block'
    option.layout.visibility = "visible"

# Panel de Opciones Lista Enlazada
btn_crearLista = Button(description='Crear Lista Enlazada', width='extended')
def on_button_crearLista_clicked(b):
    with out:
        hideShowOptions(optionsCreateList)
btn_crearLista.on_click(on_button_crearLista_clicked)

btn_addNode = Button(description='Añadir Elemento', width='extended')
def on_button_addNode_clicked(b):
    with out:
        hideShowOptions(optionsAddNode)
btn_addNode.on_click(on_button_addNode_clicked)

btn_deleteNode = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNode)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Existe Elemento', width='extended')
def on_button_findNode_clicked(b):
    with out:
        hideShowOptions(optionsFindNode)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNode)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_findAllNodes = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodes_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodes)
btn_findAllNodes.on_click(on_button_findAllNodes_clicked)

# Panel de Opciones Arbol
btn_crearArbol = Button(description='Crear Árbol Binario', width='extended')
def on_button_crearArbol_clicked(b):
    with out:
        hideShowOptions(optionsCreateArbol)
btn_crearArbol.on_click(on_button_crearArbol_clicked)

btn_addNodeArbol = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeArbol)
btn_addNodeArbol.on_click(on_button_addNodeArbol_clicked)

btn_deleteNodeArbol = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeArbol)
btn_deleteNodeArbol.on_click(on_button_deleteNodeArbol_clicked)

btn_findNodeArbol = Button(description='Existe Elemento', width='extended')
def on_button_findNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeArbol)
btn_findNodeArbol.on_click(on_button_findNodeArbol_clicked)

btn_findAdjNodeArbol = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeArbol)
btn_findAdjNodeArbol.on_click(on_button_findAdjNodeArbol_clicked)

btn_findAllNodesArbol = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodesArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodesArbol)
btn_findAllNodesArbol.on_click(on_button_findAllNodesArbol_clicked)

# Panel de Opciones Grafo
btn_crearGrafo = Button(description='Crear Grafo', width='extended')
def on_button_crearGrafo_clicked(b):
    with out:
        hideShowOptions(optionsCreateGrafo)
btn_crearGrafo.on_click(on_button_crearGrafo_clicked)

btn_addNodeGrafo = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeGrafo)
btn_addNodeGrafo.on_click(on_button_addNodeGrafo_clicked)

btn_deleteNodeGrafo = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeGrafo)
btn_deleteNodeGrafo.on_click(on_button_deleteNodeGrafo_clicked)

btn_findNodeGrafo = Button(description='Existe Elemento', width='extended')
def on_button_findNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeGrafo)
btn_findNodeGrafo.on_click(on_button_findNodeGrafo_clicked)

btn_findAdjNodeGrafo = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeGrafo)
btn_findAdjNodeGrafo.on_click(on_button_findAdjNodeGrafo_clicked)

### Build Opciones

In [190]:
layout_canvas = Layout(height='550px', border='1px dotted blue', overflow ='auto')
layout_options = Layout(height='auto', width='auto')#, border='1px dotted blue')

buttons_listas = [
    Box([btn_crearLista], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
    Box([btn_findAllNodes], layout=form_item_layout),
]
buttons_arboles = [
    Box([btn_crearArbol], layout=form_item_layout),
    Box([btn_addNodeArbol], layout=form_item_layout),
    Box([btn_deleteNodeArbol], layout=form_item_layout),
    Box([btn_findNodeArbol], layout=form_item_layout),
    Box([btn_findAdjNodeArbol], layout=form_item_layout),
    Box([btn_findAllNodesArbol],layout=form_item_layout)
]
buttons_grafos = [
    Box([btn_crearGrafo], layout=form_item_layout),
    Box([btn_addNodeGrafo], layout=form_item_layout),
    Box([btn_deleteNodeGrafo], layout=form_item_layout),
    Box([btn_findNodeGrafo], layout=form_item_layout),
    Box([btn_findAdjNodeGrafo], layout=form_item_layout),
]

# Cargar Estructura Lista
btn_cargarEstructuraLista = Button(description='Cargar Lista', width='extended', button_style='warning')
def on_button_cargarEstructuraLista_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "Problema al cargar la estructura: No se seleccionó ningún archivo"
            else:
                e = "Problema al cargar la estructura: " + src_path
            print(e)
        buttonsListas.layout.visibility = 'visible'
        buttonsListas.layout.display = 'block'
btn_cargarEstructuraLista.on_click(on_button_cargarEstructuraLista_clicked)

buttons_cargarEstructurasLista = [
    Box([btn_cargarEstructuraLista], layout=form_item_layout),
]

# Cargar Estructura BST
btn_cargarEstructuraBST = Button(description='Cargar BST', width='extended', button_style='warning')
def on_button_cargarEstructuraBST_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "Problema al cargar la estructura: No se seleccionó ningún archivo"
            else:
                e = "Problema al cargar la estructura: " + src_path
            print(e)
        buttonsArboles.layout.visibility = 'visible'
        buttonsArboles.layout.display = 'block'
btn_cargarEstructuraBST.on_click(on_button_cargarEstructuraBST_clicked)

buttons_cargarEstructurasBST = [
    Box([btn_cargarEstructuraBST], layout=form_item_layout),
]


buttonsCargarLista = Box(buttons_cargarEstructurasLista, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsCargarBST = Box(buttons_cargarEstructurasBST, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsListas = Box(buttons_listas, layout=Layout(
    #display='flex',
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsArboles = Box(buttons_arboles, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsGrafos = Box(buttons_grafos, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

canvas = HBox([out], layout=layout_canvas)
optionsListas = VBox([buttonsCargarLista,
                      buttonsListas,
                      optionsCreateList,
                      optionsAddNode,
                      optionsDeleteNode,
                      optionsFindNode,
                      optionsFindAdjNode,
                      optionsFindAllNodes],
                      layout=layout_options)
optionsArboles = VBox([buttonsCargarBST,
                       buttonsArboles,
                        optionsCreateArbol,
                        optionsAddNodeArbol,
                        optionsDeleteNodeArbol,
                        optionsFindNodeArbol,
                        optionsFindAdjNodeArbol,
                        optionsFindAllNodesArbol],
                        layout=layout_options)
optionsGrafos = VBox([buttonsGrafos,            
                    optionsCreateGrafo,
                    optionsAddNodeGrafo,
                    optionsDeleteNodeGrafo,
                    optionsFindNodeGrafo,
                    optionsFindAdjNodeGrafo],
                    layout=layout_options)

## Construir App

In [191]:
with out1:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsListas,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','550px',1],
          footer=None)
    display(app)
with out2:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsArboles,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','550px',1],
          footer=None)
    display(app)
with out3:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsGrafos,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','550px',1],
          footer=None)
    display(app)

display(tab) 